# PreProcessing 
sumber : https://yunusmuhammad007.medium.com/text-preprocessing-menggunakan-pandas-nltk-dan-sastrawi-untuk-large-dataset-5fb3c0a88571
data merupakan data tiktok yang diambil melalui proses scraping pada google play

In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel(r"C:\Users\user\Downloads\data_tiktok1 zet (1).xlsx")
df

C:\Users\user\AppData\Roaming\Python\Python310\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\user\AppData\Roaming\Python\Python310\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,USERNAME,SCORE,AT,komentar,KATEGORI
0,Jihan Azalia,5,2022-07-22 10:22:25,Bagus banget,POSITIF
1,Nur Aisyah,5,2022-07-22 10:22:14,Bagus bangettt.emang bagus,POSITIF
2,Sar Mila,4,2022-07-22 10:19:38,Aplikasi ini bagus tapi boros kuota,NETRAL
3,Dewi Murni,5,2022-07-22 10:18:52,Aku suka tiktok soalnya seru banget,POSITIF
4,Martabak populer,5,2022-07-22 10:16:11,keren,POSITIF
...,...,...,...,...,...
3732,Dion Vitroy,5,2022-07-19 15:56:46,Keren,POSITIF
3733,Afika Dwi azz,5,2022-07-19 15:56:39,Jos,POSITIF
3734,bangraster_ GM,5,2022-07-19 15:54:48,Lumayan seru bisa menghibur dan bertemn bnyk,POSITIF
3735,Oppo Maret,4,2022-07-19 15:52:42,"Mantap ni aplikasi ny,, bintang POSITIF Dulu ya",POSITIF


In [2]:
 #case folding
# gunakan fungsi Series.str.lower() pada Pandas
df['komentar'] = df['komentar'].str.lower()
print('Case Folding Result : \n')
print(df['komentar'].head(5))
print('\n\n\n')

Case Folding Result : 

0                           bagus banget
1             bagus bangettt.emang bagus
2    aplikasi ini bagus tapi boros kuota
3    aku suka tiktok soalnya seru banget
4                                  keren
Name: komentar, dtype: object






In [3]:
# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
import re

# ------ Tokenizing ---------

def remove_comments_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
df['komentar'] = df['komentar'].apply(remove_comments_special)

In [4]:
#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['komentar'] = df['komentar'].apply(remove_number)

In [5]:
#remove punctuation
import string
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df['komentar'] = df['komentar'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['komentar'] = df['komentar'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['komentar'] = df['komentar'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['komentar'] = df['komentar'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['komentar_tokens'] = df['komentar'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(df['komentar_tokens'].head())
print('\n\n\n')


Tokenizing Result : 

0                               [bagus, banget]
1                 [bagus, bangetttemang, bagus]
2    [aplikasi, ini, bagus, tapi, boros, kuota]
3    [aku, suka, tiktok, soalnya, seru, banget]
4                                       [keren]
Name: komentar_tokens, dtype: object






In [6]:
#stopword (filtering)
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah' ,'ter'])

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['ulasan_tokens_WSW'] = df['komentar_tokens'].apply(stopwords_removal) 


print(df['ulasan_tokens_WSW'].head())


0                    [bagus, banget]
1      [bagus, bangetttemang, bagus]
2    [aplikasi, bagus, boros, kuota]
3       [suka, tiktok, seru, banget]
4                            [keren]
Name: ulasan_tokens_WSW, dtype: object


In [7]:
#tahap normalisasi
normalizad_word = pd.read_excel(r"C:\Users\user\Documents\normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df['komentar_normalized'] = df['ulasan_tokens_WSW'].apply(normalized_term)

df['komentar_normalized'].head(10)


0                    [bagus, banget]
1      [bagus, bangetttemang, bagus]
2    [aplikasi, bagus, boros, kuota]
3       [suka, tiktok, seru, banget]
4                            [keren]
5         [mantul, pokoknya, tiktok]
6                           [puasss]
7                    [bintang, ajah]
8                     [bagus, keren]
9                            [bagus]
Name: komentar_normalized, dtype: object

In [8]:
#stemming/stemmer

# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['komentar_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['komentar_tokens_stemmed'] = df['komentar_normalized'].swifter.apply(get_stemmed_term)
print(df['komentar_tokens_stemmed'])


3986
------------------------
bagus : bagus
banget : banget
bangetttemang : bangetttemang
aplikasi : aplikasi
boros : boros
kuota : kuota
suka : suka
tiktok : tiktok
seru : seru
keren : keren
mantul : mantul
pokoknya : pokok
puasss : puasss
bintang : bintang
ajah : ajah
uh : uh
amett : amett
membagong : bagong
jamin : jamin
aman : aman
memboroskan : boros
pulsa : pulsa
good : good
berkesan : kesan
orang : orang
sayang : sayang
perhatian : perhati
bantu : bantu
gmna : gmna
susah : susah
instal : instal
anjy : anjy
sy : sy
tiktokan : tiktokan
tik : tik
tok : tok
ngetop : ngetop
deh : deh
oke : oke
dont : dont
like : like
your : your
aplicassion : aplicassion
because : because
very : very
lag : lag
menarik : tarik
mohon : mohon
perbaiki : baik
kreator : kreator
muncul : muncul
tulisan : tulis
tolong : tolong
ok : ok
bissa : bissa
download : download
pliss : pliss
kasi : kasi
mantap : mantap
update : update
video : video
kualitas : kualitas
low : low
burem : burem
udah : udah
pakai : pakai

memenuhi : penuh
syarat : syarat
sebel : sebel
updatenya : updatenya
bangetttttttttttttttttt : bangetttttttttttttttttt
apknya : apknya
bt : bt
nemu : nemu
lucu : lucu
aneh : aneh
terpasang : pasang
salah : salah
memuaskan : muas
ketinggalan : tinggal
login : login
kesal : kesal
nggunain : nggunain
tanggal : tanggal
lahir : lahir
toloong : toloong
min : min
musicnya : musicnya
semalam : malam
perbarui : baru
dibenahi : benah
lur : lur
ih : ih
ni : ni
blokirnyasi : blokirnyasi
nekad : nekad
menu : menu
knl : knl
dunia : dunia
busetttttt : busetttttt
apkkk : apkkk
en : en
madura : madura
tolol : tolol
bangetttttt : bangetttttt
bahasa : bahasa
inggris : inggris
skarang : skarang
shop : shop
salsa : salsa
septiani : septiani
ningsih : ningsih
film : film
bagussssssssss : bagussssssssss
bangettttttttttttttt : bangettttttttttttttt
kesel : kesel
bgtefek : bgtefek
zoom : zoom
face : face
bahus : bahus
yh : yh
kembalikan : kembali
menginstal : menginstal
lamakalo : lamakalo
mah : mah
cepat : cep

mengerti : erti
dahh : dahh
bguss : bguss
ngentod : ngentod
nda : nda
ngirim : ngirim
skli : skli
tiktokkku : tiktokkku
atur : atur
volume : volume
aq : aq
rindu : rindu
gaya : gaya
mu : mu
ranjang : ranjang
wh : wh
berbau : bau
diemin : diemin
woilahh : woilahh
niatnya : niat
diblokir : blokir
jum : jum
jumiati : jumiati
recomend : recomend
jelasleleeeeeeeeeeeeeetttttttttttt : jelasleleeeeeeeeeeeeeetttttttttttt
fuuuuulllllllll : fuuuuulllllllll
menggagu : gagu
gada : gada
fto : fto
tangguh : tangguh
pelanggaran : langgar
usia : usia
umurmasa : umurmasa
adek : adek
akses : akses
slma : slma
thga : thga
akal : akal
bangetudah : bangetudah
ajukan : aju
umpan : umpan
smpe : smpe
kabar : kabar
dri : dri
tiktoksaya : tiktoksaya
ajuin : ajuin
respontiktok : respontiktok
lamanegatif : lamanegatif
kalou : kalou
menonton : tonton
sultan : sultan
trnyata : trnyata
manpa : manpa
sebagus : bagus
gc : gc
juan : juan
bagustapi : bagustapi
diupdet : diupdet
samabelum : samabelum
bisajualan : bisajual

membantutetapi : membantutetapi
kendala : kendala
pause : pause
tergantung : gantung
sound : sound
jugatetapi : jugatetapi
membebankan : beban
sayadan : sayadan
krna : krna
hpus : hpus
alasan : alas
umurmengapa : umurmengapa
memangnya : memang
jogetnya : joget
melebihi : lebih
batas : batas
bberapa : bberapa
serupamemakai : serupamemakai
baju : baju
sexy : sexy
ngg : ngg
sepeleh : peleh
instalnya : instalnya
ngelag : ngelag
wahhhh : wahhhh
ha : ha
aplnya : aplnya
buatt : buatt
menghiburr : menghiburr
jiwa : jiwa
star : star
dulu : dulu
semnjak : semnjak
bedanya : beda
skr : skr
jaringn : jaringn
wiffi : wiffi
harusny : harusny
penggemar : gemar
hrsny : hrsny
bersyukur : syukur
sungguh : sungguh
tingkat : tingkat
ancak : ancak
tala : tala
sagat : sagat
konsisten : konsisten
jaga : jaga
blum : blum
cair : cair
dowloaad : dowloaad
soo : soo
buru : buru
unduh : unduh
penggunduhan : penggunduhan
gagal : gagal
membagongkan : bagong
ulasan : ulas
sumgguh : sumgguh
kemari : kemari
kebnyakan : 

dilaporin : dilaporin
hi : hi
lek : lek
suskreb : suskreb
sumber : sumber
aflikasi : aflikasi
daya : daya
noral : noral
ma : ma
jeles : jeles
yess : yess
bkefnya : bkefnya
dewasa : dewasa
banggek : banggek
solan : sol
macccceeeeeeeeetttttttttttttttttttttt : macccceeeeeeeeetttttttttttttttttttttt
oyt : oyt
jalan : jalan
inyal : inyal
nalama : nalama
sbelumnya : sbelumnya
ngedownload : ngedownload
jf : jf
eih : eih
pllll : pllll
lllnegatif : lllnegatif
wi : wi
fi : fi
setabil : setabil
seluler : seluler
bgggt : bgggt
sekalit : sekalit
bagut : bagut
rada : rada
beres : beres
kenak : nak
kolong : kolong
fuso : fuso
kokkkkkkkkkkkkkkkkkkkkkkkk : kokkkkkkkkkkkkkkkkkkkkkkkk
dadah : dadah
byyyyyyyyytttfsysbsnkshaga : byyyyyyyyytttfsysbsnkshaga
mobile : mobile
legends : legends
nabil : nabil
denis : den
ene : ene
ena : ena
gasi : gas
sex : sex
banned : banned
gaj : gaj
novianghua : novianghua
wwwwwwwwwwwwwwwwwww : wwwwwwwwwwwwwwwwwww
aaaaaaaaaaaaaaaaaaaaaaaaa : aaaaaaaaaaaaaaaaaaaaaaaaa
taikondo 

...

ngerti : ngerti
ker : ker
dateng : dateng
nyatapi : nyatapi
adekku : adekku
bner : bner
buriktapi : buriktapi
hayuk : hayuk
pale : pale
papa : papa
digi : digi
bam : bam
kesininegatif : kesininegatif
dibikin : bikin
keatas : atas
salin : salin
iri : iri
dengki : dengki
merspon : merspon
kontenku : konten
hukuman : hukum
negatifribu : negatifribu
jhon : jhon
plissss : plissss
oi : oi
priktrend : priktrend
kejsbsjsbjshsntrus : kejsbsjsbjshsntrus
cwo : cwo
nampilin : nampilin
sesuai : sesuai
kenyataannyabisasalah : kenyataannyabisasalah
paham : paham
sekalu : sekalu
stiker : stiker
trim : trim
kelompok : kelompok
tertentuporno : tertentuporno
kebencian : benci
diblokirdan : diblokirdan
seseorangulamatakun : seseorangulamatakun
lenyap : lenyap
sich : sich
laporan : lapor
tanggapi : tanggap
ajibb : ajibb
akunseperti : akunseperti
nowhat : nowhat
problem : problem
noob : noob
stun : stun
year : year
world : world
ngelegpokoknya : ngelegpokoknya
normal : normal
omg : omg
repost : repost
gegew

bangetttttttt : bangetttttttt
gem : gem
enya : enya
langgar : langgar
biasanegatif : biasanegatif
user : user
debes : debes
cena : cena
cek : cek
nandafail : nandafail
undang : undang
tapinya : tapi
maksudnya : maksud
allhamdulilah : allhamdulilah
sehat : sehat
bismillah : bismillah
kurah : kurah
lelalu : lalu
hek : hek
temanjangan : temanjangan
blng : blng
lamah : lamah
selep : selep
wkw : wkw
namatin : namatin
coin : coin
berjalan : jalan
cion : cion
adagimana : adagimana
mafia : mafia
yahati : yahati
nie : nie
elek : elek
kebocor : bocor
kebongkar : bongkar
daunlot : daunlot
gaming : gaming
remek : rek
arin : arin
marahin : marahin
memasuk : masuk
bersilahturahmi : bersilahturahmi
mtp : mtp
super : super
dimas : mas
fropesional : fropesional
sonal : sonal
slebew : slebew
keluarin : keluarin
aloo : aloo
tikntik : tikntik
mag : mag
sud : sud
mamang : mamang
bagusssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssesswsekalaisssssssssekaliaaaaaaaaaaaaaaaaaeaeaea

heang : heang
juragan : juragan
sekrng : sekrng
enggk : enggk
sesusah : susah
skrngsya : skrngsya
pngguna : pngguna
gnitolong : gnitolong
viewernya : viewernya
wish : wish
had : had
voice : voice
calling : calling
feature : feature
gf : gf
next : next
only : only
left : left
pcran : pcran
temenan : temenan
dibenerin : dibenerin
dijamin : jamin
minusnya : minus
cwek : cwek
gimn : gimn
yo : yo
menit : menit
cobah : cobah
gambarnya : gambar
liveku : liveku
vtku : vtku
liveeh : liveeh
lumaynn : lumaynn
woiii : woiii
ulangnyaaa : ulangnyaaa
ilanggg : ilanggg
ampuuunn : ampuuunn
dipaket : paket
income : income
masukmasuk : masukmasuk
terhiburlah : hibur
mengupload : mengupload
jernih : jernih
berkualitas : kualitas
opsi : opsi
kompres : kompres
stop : stop
seksi : seksi
terpopuler : populer
tukar : tukar
positifrb : positifrb
ilanng : ilanng
asalamualaikum : asalamualaikum
wr : wr
wb : wb
alot : alot
emot : emot
monyet : monyet
ganteng : ganteng
jahat : jahat
toktok : toktok
gilah : gilah
mu

Pandas Apply:   0%|          | 0/3737 [00:00<?, ?it/s]

0                                [bagus, banget]
1                  [bagus, bangetttemang, bagus]
2                [aplikasi, bagus, boros, kuota]
3                   [suka, tiktok, seru, banget]
4                                        [keren]
                          ...                   
3732                                     [keren]
3733                                       [jos]
3734       [lumayan, seru, hibur, bertemn, bnyk]
3735    [mantap, ni, aplikasi, bintang, positif]
3736                                       [oke]
Name: komentar_tokens_stemmed, Length: 3737, dtype: object


In [9]:
#save data
df.to_excel("tiktok.xlsx")